 GNews API

In [ ]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 6.0 MB/s eta 0:00:00


In [ ]:
import os
import boto3
from botocore.config import Config
from botocore import UNSIGNED
import json
import urllib.request
import pandas as pd
from google.colab import userdata
import datetime  # For date-based naming
import json

In [ ]:
TEAM = "TEAM_5/"

BUCKET_NAME = "cus635-spring2025"

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

api_key = userdata.get("API_KEY")


In [ ]:
#grab the date of upload to s3 which happens at runtime
current_date = datetime.datetime.now().strftime("%m-%d-%Y")  # Format: MM-DD-YYYY

#Gnews Api vars
src = "gnews.io"
q_topic = "World News"
#need to override qtopic for world news
q_topic = "world"
q_topic_encoded = urllib.parse.quote(q_topic)
lang = "en"
country = "us"
news_return_maximum = "10"

#Set Url based on Gnews Api params
url = f"https://gnews.io/api/v4/search?q={q_topic_encoded}&lang={lang}&country={country}&max={news_return_maximum}&apikey={api_key}"
#search by category with top headlines
#url = f"https://gnews.io/api/v4/top-headlines?category={q_topic_encoded}&lang={lang}&country={country}&max={news_return_maximum}&apikey={api_key}"


In [ ]:
with urllib.request.urlopen(url) as response:
    data = json.loads(response.read().decode("utf-8"))
    articles = data["articles"]

#This function will take each article from article list and upload the object under TEAM category with the key
#being a formatted object name with a truncated title of 15 chars for standardization
#body is the json data from the from the articles list at index i
    for i in range(len(articles)):
        article_data = {
            "title": articles[i]["title"],
            "description": articles[i]["description"],
            "content": articles[i]["content"],
            "url": articles[i]["url"],
            "image": articles[i]["image"],
            "publishedAt": articles[i]["publishedAt"],
            "source": {
                "name": articles[i]["source"]["name"],
                "url": articles[i]["source"]["url"]
            }
        }

        # Url from article_data : Replace object name : user preference
        url_article = article_data['url']

        # Truncate title to prevent giant_blob_of_text.jsons
        title = article_data['title'][:15] + "..." if len(article_data['title']) > 15 else article_data['title']

        # Create a unique filename based on the date
        object_name = f"src_{src}_{q_topic}_retrieved_{current_date}_{i + 1}_article_{title}.json"

        # Upload the article to S3 (stored as a JSON file with a unique name)
        s3.put_object(
            Bucket=BUCKET_NAME,
            Key=f"{TEAM}{object_name}",
            Body=json.dumps(article_data),
            ContentType='application/json'
        )

        print(f"Successfully uploaded {object_name} to {BUCKET_NAME}/{TEAM}{object_name}")

Successfully uploaded src_gnews.io_world_retrieved_03-11-2025_1_article_World's largest....json to cus635-spring2025/TEAM_5/src_gnews.io_world_retrieved_03-11-2025_1_article_World's largest....json
Successfully uploaded src_gnews.io_world_retrieved_03-11-2025_2_article_Barcelona teena....json to cus635-spring2025/TEAM_5/src_gnews.io_world_retrieved_03-11-2025_2_article_Barcelona teena....json
Successfully uploaded src_gnews.io_world_retrieved_03-11-2025_3_article_Dodgers’ Boss W....json to cus635-spring2025/TEAM_5/src_gnews.io_world_retrieved_03-11-2025_3_article_Dodgers’ Boss W....json
Successfully uploaded src_gnews.io_world_retrieved_03-11-2025_4_article_You'll Have Per....json to cus635-spring2025/TEAM_5/src_gnews.io_world_retrieved_03-11-2025_4_article_You'll Have Per....json
Successfully uploaded src_gnews.io_world_retrieved_03-11-2025_5_article_Scottie Scheffl....json to cus635-spring2025/TEAM_5/src_gnews.io_world_retrieved_03-11-2025_5_article_Scottie Scheffl....json
Successful

In [ ]:
#AccessToS3Bucket.ipynb Professor Provision
response = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=TEAM)
if "Contents" in response:
    print("Files in S3 Bucket:")
    for obj in response["Contents"]:
        print(f"- {obj['Key']}")
else:
    print("No files found in the bucket.")


Files in S3 Bucket:
- TEAM_5/
- TEAM_5/'It’s A Mess.' Sony Was Only Supposed To Finance And Distribute It Ends With Us. How It Ended Up Playing Mediator Between Blake Lively And Justin Baldoni
- TEAM_5/'Our economy is built to withstand': B.C. offers tariff reassurance
- TEAM_5/Behind the bombshell end of the criminal case against Eric Adams
- TEAM_5/Fairfield Board Of Selectmen Restores $2 Million To Education Budget
- TEAM_5/February jobs report: Finance and big data fuel Dallas-Fort Worth growth
- TEAM_5/Ondo Finance Broadens European Reach with Tokenized US Treasuries Launch on Assetera
- TEAM_5/OpenAI CFO on possibility of going public, Musk takeover bid
- TEAM_5/Speaker Dustin Burrows promises school choice will pass: ‘Our moment has arrived’
- TEAM_5/Stacey Abrams-tied voting organization targeted in new investigation
- TEAM_5/Stocks to watch this week
- TEAM_5/article_03-07-2025_1.json
- TEAM_5/article_03-07-2025_10.json
- TEAM_5/article_03-07-2025_2.json
- TEAM_5/article_03-07

In [ ]:
#This code block is used to display the data of a json file. //this is currently hardcoded
file_key = f"""TEAM_5/src_gnews.io_World News_retrieved_03-11-2025_10_article_Lamenting Luka ....json"""

try:
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=file_key)
    json_data = json.loads(obj['Body'].read().decode('utf-8'))

    # Now you can work with the JSON data
    print(json_data)  # Print the entire JSON data
    # Or access specific elements:
    print(json_data['title'])  # Assuming 'title' is a key in your JSON

except Exception as e:
    print(f"Error: Could not retrieve or parse JSON data from {file_key}: {e}")

{'title': 'Lamenting Luka Doncic Suffered Two Financial Blows in Seconds as Painful Details of Lakers Trade Emerge', 'description': 'As the world takes in the news that Doncic is a Laker, news of him being kept in the dark surfaces as he suffers multiple financial blows.', 'content': 'Luka Doncic spent the first seven years of his NBA career suiting up for the Dallas Mavericks. The 25-year-old was a fan favorite and the face of the franchise in Dallas. Trading him almost seemed impossible until it did happen! On Saturday Doncic wa... [547 chars]', 'url': 'https://www.essentiallysports.com/nba-active-basketball-news-lamenting-luka-doncic-suffered-two-financial-blows-in-seconds-as-painful-details-of-lakers-trade-emerge/', 'image': 'https://image-cdn.essentiallysports.com/wp-content/uploads/USATSI_24732326-1-465x315.jpg', 'publishedAt': '2025-02-03T18:44:55Z', 'source': {'name': 'Essentially Sports', 'url': 'https://www.essentiallysports.com'}}
Lamenting Luka Doncic Suffered Two Financial

In [ ]:
#This code block is used to delete a json file. //this is currently hardcoded

#Not Allowed: Access Denied by Professor

file_key = f"""TEAM_5/article_Donald Trump and Elon Musk's Fox News Interview Fact Checked_category_World News_date_retrieved_03-08-2025_src_gnews.io_4.json"""

try:
    s3.delete_object(Bucket=BUCKET_NAME, Key=file_key)
    print(f"Object '{file_key}' deleted successfully.")
except Exception as e:
    print(f"Error deleting object '{file_key}': {e}")

Error deleting object 'TEAM_5/article_Donald Trump and Elon Musk's Fox News Interview Fact Checked_category_World News_date_retrieved_03-08-2025_src_gnews.io_4.json': An error occurred (AccessDenied) when calling the DeleteObject operation: Access Denied


In [ ]:
#UNUSED
#change later to fetch then upload to s3 bucket
def fetch_news(url):
    try:
        with urllib.request.urlopen(url) as response:
            data = json.loads(response.read().decode("utf-8"))
            articles = data["articles"]
            for i in range(len(articles)):
                print(f"Title: {articles[i]['title']}")
                print(f"Description: {articles[i]['description']}")
                print(f"Url: {articles[i]['url']}")
    except urllib.error.HTTPError as e:
        if e.code == 400:
            print("Error 400: Bad Request - Your request is invalid.")
        elif e.code == 401:
            print("Error 401: Unauthorized - Your API key is wrong.")
        elif e.code == 403:
            print("Error 403: Forbidden - You have reached your daily quota.")
        elif e.code == 429:
            print("Error 429: Too Many Requests - Slow down your requests.")
        elif e.code == 500:
            print("Error 500: Internal Server Error - Try again later.")
        elif e.code == 503:
            print("Error 503: Service Unavailable - Try again later.")
        else:
            print(f"An unexpected error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


fetch_news(url)